In [1]:
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc, get_team_ratings
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [82]:
pd.set_option('display.max_columns', None)  # Show all columns

#### Import and Clean Team Statistic Data

In [67]:
per_game = pd.read_csv('per_game.csv')

In [74]:
per_game["Team"] = per_game["Team"].str.replace("*", "", regex=False)

nba_teams = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

per_game['Team'] = per_game['Team'].replace(nba_teams)

per_game

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,CLE,82,240.9,44.5,90.8,0.491,15.9,41.5,0.383,28.6,49.3,0.581,17.0,21.9,0.776,11.2,34.2,45.4,28.1,8.2,4.3,13.2,18.1,121.9
1,2.0,MEM,82,240.3,44.8,93.3,0.479,13.9,37.9,0.367,30.8,55.5,0.556,18.3,23.3,0.786,12.9,34.4,47.3,28.4,8.9,5.6,15.7,20.9,121.7
2,3.0,DEN,82,242.1,45.4,89.8,0.506,12.0,31.9,0.376,33.4,57.8,0.578,17.9,23.3,0.770,11.2,34.5,45.7,31.0,8.0,4.9,14.3,17.6,120.8
3,4.0,OKC,82,240.3,44.6,92.7,0.482,14.5,38.8,0.374,30.1,53.9,0.559,16.7,20.4,0.819,10.6,34.2,44.8,26.9,10.3,5.7,11.7,19.9,120.5
4,5.0,ATL,82,241.2,43.4,91.8,0.472,13.5,37.7,0.358,29.9,54.1,0.552,18.0,23.2,0.775,11.9,32.6,44.5,29.6,9.7,5.1,15.5,19.1,118.2
5,6.0,CHI,82,240.9,43.2,92.0,0.470,15.4,42.0,0.367,27.8,50.0,0.556,16.0,19.7,0.809,10.1,35.8,45.9,29.1,7.6,4.7,14.7,17.6,117.8
6,7.0,IND,82,242.1,43.6,89.3,0.488,13.2,35.8,0.368,30.4,53.6,0.568,17.0,21.6,0.789,9.2,32.7,41.8,29.2,8.5,5.5,13.2,18.7,117.4
7,8.0,BOS,82,241.8,41.6,90.0,0.462,17.8,48.2,0.368,23.9,41.8,0.571,15.3,19.1,0.799,11.4,33.9,45.3,26.1,7.2,5.5,11.9,15.9,116.3
8,9.0,NYK,82,242.4,43.3,89.2,0.486,12.6,34.1,0.369,30.8,55.1,0.558,16.5,20.7,0.800,10.9,31.8,42.6,27.5,8.2,4.0,13.3,17.2,115.8
9,10.0,SAC,82,242.4,43.0,90.1,0.478,12.6,35.2,0.357,30.5,54.9,0.555,17.1,21.2,0.806,11.0,33.2,44.2,26.5,7.6,4.4,13.3,18.9,115.7


In [76]:
##advanced stats
team_adv = pd.read_csv('team_adv.csv')
team_adv

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,Unnamed: 17,eFG%,TOV%,ORB%,FT/FGA,Unnamed: 22,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Unnamed: 27,Arena,Attend.,Attend./G
0,1.0,Oklahoma City Thunder*,24.8,68.0,14.0,68,14,12.87,-0.16,12.70,120.3,107.5,12.8,100.0,0.220,0.419,0.593,NaN,0.560,10.3,24.2,0.180,NaN,0.513,14.9,74.6,0.211,NaN,Paycom Center,754832,17973
1,2.0,Boston Celtics*,28.9,61.0,21.0,62,20,9.11,-0.83,8.28,120.6,111.1,9.5,95.7,0.212,0.536,0.591,NaN,0.561,10.8,25.7,0.169,NaN,0.522,11.6,76.0,0.154,NaN,TD Garden,785396,19156
2,3.0,Cleveland Cavaliers*,26.6,64.0,18.0,62,20,9.54,-0.73,8.81,121.7,112.2,9.5,99.8,0.241,0.457,0.607,NaN,0.578,11.6,25.9,0.187,NaN,0.528,12.6,74.8,0.181,NaN,Rocket Arena,796712,19432
3,4.0,Minnesota Timberwolves*,27.2,49.0,33.0,53,29,5.00,0.15,5.15,116.6,111.5,5.1,97.3,0.249,0.455,0.588,NaN,0.554,13.0,25.8,0.196,NaN,0.532,13.2,75.1,0.178,NaN,Target Center,772249,18835
4,5.0,Los Angeles Clippers*,29.7,50.0,32.0,53,29,4.66,0.18,4.84,115.1,110.3,4.8,97.5,0.251,0.387,0.589,NaN,0.554,13.4,24.4,0.200,NaN,0.536,13.7,77.5,0.189,NaN,Intuit Dome,679593,16575
5,6.0,Memphis Grizzlies*,24.7,48.0,34.0,52,30,4.85,-0.06,4.79,117.7,113.0,4.7,103.3,0.249,0.406,0.588,NaN,0.554,13.1,28.7,0.196,NaN,0.533,12.9,74.9,0.206,NaN,FedEx Forum,683067,16660
6,7.0,Houston Rockets*,24.9,52.0,30.0,52,30,4.51,0.45,4.97,115.3,110.8,4.5,98.6,0.242,0.384,0.553,NaN,0.523,11.8,31.7,0.178,NaN,0.528,12.8,76.2,0.186,NaN,Toyota Center,716853,17484
7,8.0,New York Knicks*,27.5,51.0,31.0,51,31,4.10,-0.51,3.59,118.5,114.3,4.2,96.7,0.232,0.382,0.589,NaN,0.556,11.9,26.0,0.186,NaN,0.549,13.1,74.5,0.176,NaN,Madison Square Garden (IV),811794,19800
8,9.0,Denver Nuggets*,27.0,50.0,32.0,50,32,3.89,0.08,3.97,119.9,116.0,3.9,99.8,0.259,0.356,0.604,NaN,0.573,12.5,26.7,0.200,NaN,0.542,11.3,74.6,0.173,NaN,Ball Arena,811211,19786
9,10.0,Golden State Warriors*,28.6,48.0,34.0,49,33,3.30,0.25,3.56,115.0,111.7,3.3,98.7,0.244,0.469,0.568,NaN,0.536,12.3,27.3,0.187,NaN,0.541,14.1,75.6,0.189,NaN,Chase Center,740624,18064


In [77]:
team_adv['Team'] = team_adv['Team'].str.replace("*", "", regex=False)
team_adv['Team'] = team_adv['Team'].replace(nba_teams)
team_adv

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,Unnamed: 17,eFG%,TOV%,ORB%,FT/FGA,Unnamed: 22,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Unnamed: 27,Arena,Attend.,Attend./G
0,1.0,OKC,24.8,68.0,14.0,68,14,12.87,-0.16,12.70,120.3,107.5,12.8,100.0,0.220,0.419,0.593,NaN,0.560,10.3,24.2,0.180,NaN,0.513,14.9,74.6,0.211,NaN,Paycom Center,754832,17973
1,2.0,BOS,28.9,61.0,21.0,62,20,9.11,-0.83,8.28,120.6,111.1,9.5,95.7,0.212,0.536,0.591,NaN,0.561,10.8,25.7,0.169,NaN,0.522,11.6,76.0,0.154,NaN,TD Garden,785396,19156
2,3.0,CLE,26.6,64.0,18.0,62,20,9.54,-0.73,8.81,121.7,112.2,9.5,99.8,0.241,0.457,0.607,NaN,0.578,11.6,25.9,0.187,NaN,0.528,12.6,74.8,0.181,NaN,Rocket Arena,796712,19432
3,4.0,MIN,27.2,49.0,33.0,53,29,5.00,0.15,5.15,116.6,111.5,5.1,97.3,0.249,0.455,0.588,NaN,0.554,13.0,25.8,0.196,NaN,0.532,13.2,75.1,0.178,NaN,Target Center,772249,18835
4,5.0,LAC,29.7,50.0,32.0,53,29,4.66,0.18,4.84,115.1,110.3,4.8,97.5,0.251,0.387,0.589,NaN,0.554,13.4,24.4,0.200,NaN,0.536,13.7,77.5,0.189,NaN,Intuit Dome,679593,16575
5,6.0,MEM,24.7,48.0,34.0,52,30,4.85,-0.06,4.79,117.7,113.0,4.7,103.3,0.249,0.406,0.588,NaN,0.554,13.1,28.7,0.196,NaN,0.533,12.9,74.9,0.206,NaN,FedEx Forum,683067,16660
6,7.0,HOU,24.9,52.0,30.0,52,30,4.51,0.45,4.97,115.3,110.8,4.5,98.6,0.242,0.384,0.553,NaN,0.523,11.8,31.7,0.178,NaN,0.528,12.8,76.2,0.186,NaN,Toyota Center,716853,17484
7,8.0,NYK,27.5,51.0,31.0,51,31,4.10,-0.51,3.59,118.5,114.3,4.2,96.7,0.232,0.382,0.589,NaN,0.556,11.9,26.0,0.186,NaN,0.549,13.1,74.5,0.176,NaN,Madison Square Garden (IV),811794,19800
8,9.0,DEN,27.0,50.0,32.0,50,32,3.89,0.08,3.97,119.9,116.0,3.9,99.8,0.259,0.356,0.604,NaN,0.573,12.5,26.7,0.200,NaN,0.542,11.3,74.6,0.173,NaN,Ball Arena,811211,19786
9,10.0,GSW,28.6,48.0,34.0,49,33,3.30,0.25,3.56,115.0,111.7,3.3,98.7,0.244,0.469,0.568,NaN,0.536,12.3,27.3,0.187,NaN,0.541,14.1,75.6,0.189,NaN,Chase Center,740624,18064


#### Merge Team Statistics DataFrames

In [81]:
full_ts = pd.merge(per_game, team_adv, how='inner', left_on='Team', right_on='Team')
full_ts

,Rk_x,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Rk_y,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,Unnamed: 17,eFG%,TOV%,ORB%,FT/FGA,Unnamed: 22,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Unnamed: 27,Arena,Attend.,Attend./G
0,1.0,CLE,82,240.9,44.5,90.8,0.491,15.9,41.5,0.383,28.6,49.3,0.581,17.0,21.9,0.776,11.2,34.2,45.4,28.1,8.2,4.3,13.2,18.1,121.9,3.0,26.6,64.0,18.0,62,20,9.54,-0.73,8.81,121.7,112.2,9.5,99.8,0.241,0.457,0.607,NaN,0.578,11.6,25.9,0.187,NaN,0.528,12.6,74.8,0.181,NaN,Rocket Arena,796712,19432
1,2.0,MEM,82,240.3,44.8,93.3,0.479,13.9,37.9,0.367,30.8,55.5,0.556,18.3,23.3,0.786,12.9,34.4,47.3,28.4,8.9,5.6,15.7,20.9,121.7,6.0,24.7,48.0,34.0,52,30,4.85,-0.06,4.79,117.7,113.0,4.7,103.3,0.249,0.406,0.588,NaN,0.554,13.1,28.7,0.196,NaN,0.533,12.9,74.9,0.206,NaN,FedEx Forum,683067,16660
2,3.0,DEN,82,242.1,45.4,89.8,0.506,12.0,31.9,0.376,33.4,57.8,0.578,17.9,23.3,0.770,11.2,34.5,45.7,31.0,8.0,4.9,14.3,17.6,120.8,9.0,27.0,50.0,32.0,50,32,3.89,0.08,3.97,119.9,116.0,3.9,99.8,0.259,0.356,0.604,NaN,0.573,12.5,26.7,0.200,NaN,0.542,11.3,74.6,0.173,NaN,Ball Arena,811211,19786
3,4.0,OKC,82,240.3,44.6,92.7,0.482,14.5,38.8,0.374,30.1,53.9,0.559,16.7,20.4,0.819,10.6,34.2,44.8,26.9,10.3,5.7,11.7,19.9,120.5,1.0,24.8,68.0,14.0,68,14,12.87,-0.16,12.70,120.3,107.5,12.8,100.0,0.220,0.419,0.593,NaN,0.560,10.3,24.2,0.180,NaN,0.513,14.9,74.6,0.211,NaN,Paycom Center,754832,17973
4,5.0,ATL,82,241.2,43.4,91.8,0.472,13.5,37.7,0.358,29.9,54.1,0.552,18.0,23.2,0.775,11.9,32.6,44.5,29.6,9.7,5.1,15.5,19.1,118.2,18.0,24.9,40.0,42.0,38,44,-1.13,-0.28,-1.41,114.6,115.7,-1.1,102.6,0.252,0.410,0.579,NaN,0.546,13.2,26.3,0.196,NaN,0.560,13.8,76.0,0.202,NaN,State Farm Arena,657613,16440
5,6.0,CHI,82,240.9,43.2,92.0,0.470,15.4,42.0,0.367,27.8,50.0,0.556,16.0,19.7,0.809,10.1,35.8,45.9,29.1,7.6,4.7,14.7,17.6,117.8,20.0,25.2,39.0,43.0,37,45,-1.56,-0.27,-1.83,114.1,115.6,-1.5,102.8,0.214,0.457,0.585,NaN,0.553,12.7,22.3,0.173,NaN,0.539,10.8,76.6,0.178,NaN,United Center,825659,20138
6,7.0,IND,82,242.1,43.6,89.3,0.488,13.2,35.8,0.368,30.4,53.6,0.568,17.0,21.6,0.789,9.2,32.7,41.8,29.2,8.5,5.5,13.2,18.7,117.4,12.0,25.8,50.0,32.0,46,36,2.23,-0.55,1.68,116.5,114.3,2.2,99.9,0.241,0.400,0.594,NaN,0.562,11.8,21.3,0.190,NaN,0.546,13.1,74.5,0.190,NaN,Gainbridge Fieldhouse,685434,16737
7,8.0,BOS,82,241.8,41.6,90.0,0.462,17.8,48.2,0.368,23.9,41.8,0.571,15.3,19.1,0.799,11.4,33.9,45.3,26.1,7.2,5.5,11.9,15.9,116.3,2.0,28.9,61.0,21.0,62,20,9.11,-0.83,8.28,120.6,111.1,9.5,95.7,0.212,0.536,0.591,NaN,0.561,10.8,25.7,0.169,NaN,0.522,11.6,76.0,0.154,NaN,TD Garden,785396,19156
8,9.0,NYK,82,242.4,43.3,89.2,0.486,12.6,34.1,0.369,30.8,55.1,0.558,16.5,20.7,0.800,10.9,31.8,42.6,27.5,8.2,4.0,13.3,17.2,115.8,8.0,27.5,51.0,31.0,51,31,4.10,-0.51,3.59,118.5,114.3,4.2,96.7,0.232,0.382,0.589,NaN,0.556,11.9,26.0,0.186,NaN,0.549,13.1,74.5,0.176,NaN,Madison Square Garden (IV),811794,19800
9,10.0,SAC,82,242.4,43.0,90.1,0.478,12.6,35.2,0.357,30.5,54.9,0.555,17.1,21.2,0.806,11.0,33.2,44.2,26.5,7.6,4.4,13.3,18.9,115.7,16.0,27.8,40.0,42.0,42,40,0.48,0.11,0.58,116.7,116.2,0.5,98.2,0.235,0.391,0.582,NaN,0.548,11.8,25.4,0.190,NaN,0.557,12.5,76.9,0.200,NaN,Golden 1 Center,696409,16986


In [286]:
full_ts.to_csv('full_ts.csv')

#### Pull Game Schedule Data

In [164]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

months = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
games_df = pd.DataFrame()

for month in months:
    # URL of the games page
    url = f"https://www.basketball-reference.com/leagues/NBA_2025_games-{month}.html"

    # Send a GET request
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the games table
    table = soup.find('table', id='schedule')

    # Extract column names
    columns = [th.text for th in table.find('thead').find_all('th')]

    games = []

    # Loop through table rows
    for row in table.find('tbody').find_all('tr'):
        if row.find('th', {"scope": "row"}) is None:
            continue  # Skip separator rows

        date = row.find('th', {"scope": "row"}).text  # Extract date from the <th>
        game_data = [td.text for td in row.find_all('td')]

        games.append([date] + game_data)  # Include date as first element

    # Create a DataFrame
    df = pd.DataFrame(games, columns=columns)
    games_df = pd.concat([games_df, df], ignore_index=True)


In [165]:
# Clean the column names
print(games_df.columns)

games_df.rename(columns={
    'Date': 'Date',
    'Visitor/Neutral': 'Visitor',
    'Home/Neutral': 'Home',
}, inplace=True)

# Handle duplicate 'PTS' columns
new_columns = []
counter = 0
for col in games_df.columns:
    if col == 'PTS':
        if counter == 0:
            new_columns.append('VPTS')  # Visitor Points
        else:
            new_columns.append('HPTS')  # Home Points
        counter += 1
    else:
        new_columns.append(col)

# Assign new column names back
games_df.columns = new_columns

games_df.columns


Index(['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS',
       ' ', ' ', 'Attend.', 'LOG', 'Arena', 'Notes'],
      dtype='object')


Index(['Date', 'Start (ET)', 'Visitor', 'VPTS', 'Home', 'HPTS', ' ', ' ',
       'Attend.', 'LOG', 'Arena', 'Notes'],
      dtype='object')

In [166]:
#make date column in date format
games_df['Date'] = pd.to_datetime(games_df['Date'])

In [167]:
#remove playoff games
games_df = games_df[games_df['Date'] < pd.Timestamp('2025-04-19')]

In [168]:
games_df['Outcome'] = games_df.apply(
    lambda row: 'W' if pd.to_numeric(row['HPTS'], errors='coerce') > pd.to_numeric(row['VPTS'], errors='coerce') else 'L',
    axis=1
)


In [169]:
#change team names to abbreviations
team_abbreviations = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'Charlotte Hornets': 'CHA',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}

# Replace for Home and Visitor columns
games_df['Home'] = games_df['Home'].replace(team_abbreviations)
games_df['Visitor'] = games_df['Visitor'].replace(team_abbreviations)


In [170]:
games_df

,Date,Start (ET),Visitor,VPTS,Home,HPTS,,,Attend.,LOG,Arena,Notes,Outcome
0,2024-10-22,7:30p,NYK,109,BOS,132,Box Score,,"19,156",2:04,TD Garden,,W
1,2024-10-22,10:00p,MIN,103,LAL,110,Box Score,,"18,997",2:26,Crypto.com Arena,,W
2,2024-10-23,7:00p,IND,115,DET,109,Box Score,,"20,062",2:23,Little Caesars Arena,,L
3,2024-10-23,7:30p,BKN,116,ATL,120,Box Score,,"17,548",2:33,State Farm Arena,,W
4,2024-10-23,7:30p,ORL,116,MIA,97,Box Score,,"19,630",2:31,Kaseya Center,,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,2025-04-15,10:00p,MEM,116,GSW,121,Box Score,,"18,064",2:46,Chase Center,Play-In Game,W
1233,2025-04-16,7:30p,MIA,109,CHI,90,Box Score,,"21,380",2:16,United Center,Play-In Game,L
1234,2025-04-16,10:00p,DAL,120,SAC,106,Box Score,,"17,939",2:20,Golden 1 Center,Play-In Game,L
1235,2025-04-18,7:00p,MIA,123,ATL,114,Box Score,OT,"17,690",2:35,State Farm Arena,Play-In Game,L


In [173]:
#drop unessecary columns
games_df = games_df.drop(games_df.columns[[1,6, 7, 8, 9, 10, 11]], axis=1)


#### Merge Team Stats with Game Schedule

In [263]:
#merge home team stats

home_stats = pd.merge(games_df, full_ts, how='left', left_on='Home', right_on = 'Team')

columns = ['Rk_x', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',
           'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
           'Rk_y', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg',
           'Pace', 'FTr', '3PAr', 'TS%', 'Unnamed: 17', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA',
           'Unnamed: 22', 'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1', 'Unnamed: 27', 'Arena',
           'Attend.', 'Attend./G']

# Make a dictionary to add "Home" prefix
rename_dict = {col: f'Home {col}' for col in columns}

In [264]:
home_stats = home_stats.rename(columns = rename_dict)

In [265]:
#merge away stats onto the game schedule
full_data = pd.merge(home_stats, full_ts, how = 'left', left_on = 'Visitor', right_on = 'Team')

In [266]:
full_data

,Date,Visitor,VPTS,Home,HPTS,Outcome,Home Rk_x,Home Team,Home G,Home MP,Home FG,Home FGA,Home FG%,Home 3P,Home 3PA,Home 3P%,Home 2P,Home 2PA,Home 2P%,Home FT,Home FTA,Home FT%,Home ORB,Home DRB,Home TRB,Home AST,Home STL,Home BLK,Home TOV,Home PF,Home PTS,Home Rk_y,Home Age,Home W,Home L,Home PW,Home PL,Home MOV,Home SOS,Home SRS,Home ORtg,Home DRtg,Home NRtg,Home Pace,Home FTr,Home 3PAr,Home TS%,Home Unnamed: 17,Home eFG%,Home TOV%,Home ORB%,Home FT/FGA,Home Unnamed: 22,Home eFG%.1,Home TOV%.1,Home DRB%,Home FT/FGA.1,Home Unnamed: 27,Home Arena,Home Attend.,Home Attend./G,Rk_x,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Rk_y,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,Unnamed: 17,eFG%,TOV%,ORB%,FT/FGA,Unnamed: 22,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Unnamed: 27,Arena,Attend.,Attend./G
0,2024-10-22,NYK,109,BOS,132,W,8.0,BOS,82,241.8,41.6,90.0,0.462,17.8,48.2,0.368,23.9,41.8,0.571,15.3,19.1,0.799,11.4,33.9,45.3,26.1,7.2,5.5,11.9,15.9,116.3,2.0,28.9,61.0,21.0,62,20,9.11,-0.83,8.28,120.6,111.1,9.5,95.7,0.212,0.536,0.591,NaN,0.561,10.8,25.7,0.169,NaN,0.522,11.6,76.0,0.154,NaN,TD Garden,785396,19156,9.0,NYK,82,242.4,43.3,89.2,0.486,12.6,34.1,0.369,30.8,55.1,0.558,16.5,20.7,0.800,10.9,31.8,42.6,27.5,8.2,4.0,13.3,17.2,115.8,8.0,27.5,51.0,31.0,51,31,4.10,-0.51,3.59,118.5,114.3,4.2,96.7,0.232,0.382,0.589,NaN,0.556,11.9,26.0,0.186,NaN,0.549,13.1,74.5,0.176,NaN,Madison Square Garden (IV),811794,19800
1,2024-10-22,MIN,103,LAL,110,W,19.0,LAL,82,240.6,40.9,85.5,0.479,13.3,36.4,0.366,27.6,49.1,0.562,18.2,23.2,0.785,9.7,32.8,42.4,26.0,7.7,4.5,14.0,17.3,113.4,14.0,27.9,50.0,32.0,44,38,1.22,0.23,1.45,115.9,114.7,1.2,97.6,0.271,0.425,0.593,NaN,0.557,12.7,23.3,0.213,NaN,0.539,12.2,74.5,0.178,NaN,Crypto.com Arena,767626,18723,13.0,MIN,82,241.8,41.0,87.6,0.468,15.0,39.9,0.377,26.0,47.7,0.545,17.2,21.8,0.789,11.1,33.2,44.3,26.1,8.0,5.0,14.5,18.3,114.3,4.0,27.2,49.0,33.0,53,29,5.00,0.15,5.15,116.6,111.5,5.1,97.3,0.249,0.455,0.588,NaN,0.554,13.0,25.8,0.196,NaN,0.532,13.2,75.1,0.178,NaN,Target Center,772249,18835
2,2024-10-23,IND,115,DET,109,L,12.0,DET,82,241.5,42.7,89.8,0.476,12.8,35.4,0.362,29.9,54.4,0.550,17.2,22.3,0.774,11.4,33.4,44.8,26.4,8.0,5.3,14.9,20.7,115.5,13.0,25.8,44.0,38.0,46,36,1.90,-0.17,1.73,115.0,113.1,1.9,99.8,0.248,0.394,0.580,NaN,0.547,13.0,26.2,0.192,NaN,0.539,13.3,76.0,0.214,NaN,Little Caesars Arena,781929,19071,7.0,IND,82,242.1,43.6,89.3,0.488,13.2,35.8,0.368,30.4,53.6,0.568,17.0,21.6,0.789,9.2,32.7,41.8,29.2,8.5,5.5,13.2,18.7,117.4,12.0,25.8,50.0,32.0,46,36,2.23,-0.55,1.68,116.5,114.3,2.2,99.9,0.241,0.400,0.594,NaN,0.562,11.8,21.3,0.190,NaN,0.546,13.1,74.5,0.190,NaN,Gainbridge Fieldhouse,685434,16737
3,2024-10-23,BKN,116,ATL,120,W,5.0,ATL,82,241.2,43.4,91.8,0.472,13.5,37.7,0.358,29.9,54.1,0.552,18.0,23.2,0.775,11.9,32.6,44.5,29.6,9.7,5.1,15.5,19.1,118.2,18.0,24.9,40.0,42.0,38,44,-1.13,-0.28,-1.41,114.6,115.7,-1.1,102.6,0.252,0.410,0.579,NaN,0.546,13.2,26.3,0.196,NaN,0.560,13.8,76.0,0.202,NaN,State Farm Arena,657613,16440,30.0,BKN,82,241.2,37.6,86.1,0.437,13.6,39.4,0.344,24.1,46.7,0.515,16.3,20.7,0.787,10.9,30.4,41.3,25.2,7.8,4.3,15.2,20.7,105.1,26.0,24.7,26.0,56.0,23,59,-7.11,0.16,-6.95,108.5,115.8,-7.3,96.4,0.240,0.458,0.552,NaN,0.516,13.7,24.5,0.189,NaN,0.557,13.7,75.0,0.218,NaN,Barclays Center,713304,17398
4,2024-10-23,ORL,116,MIA,97,L,24.0,MIA,82,243.0,40.5,87.0,0.465,13.7,37.3,0.367,26.8,49.7,0.538,16.0,20.2,0.791,9.7,33.7,43.4,26.4,8.1,3.8,13.6,15.5,110.6,15.0,26.8,37.0,45.0,42,40,0.56,-0.45,0.11,113.5,112.9,0.6,96.2,0.232,0.429,0.576,NaN,0.544,12.4,22.3,0.183,NaN,0.543,12.5,75.8,0.157,NaN,Kaseya Center,808337,19716,28.0,ORL,82,240.6,38.2,85.8,0.445,11.2,35.3,0.318,26.9,50.5,0.533,17.9,23.0,0.775,11.1,30.7,41.8,23.0,8.9,6.0,14.2,20.1,105.4,17.0,24.9,41.0,41.0,41,41,-0.12,-0.58,-0.70,109.4,109.6,-0.2,96.1,0.269,0.411,0.550,NaN,0.510,12.9,25.3,0.208,NaN,0.538,15.0,77.0,0.220,NaN,Kia Center,762121,18588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [267]:
#clean away columns names
columns = ['Rk_x', 'Team', 'G', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%',
           'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
           'Rk_y', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS', 'ORtg', 'DRtg', 'NRtg',
           'Pace', 'FTr', '3PAr', 'TS%', 'Unnamed: 17', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA',
           'Unnamed: 22', 'eFG%.1', 'TOV%.1', 'DRB%', 'FT/FGA.1', 'Unnamed: 27', 'Arena',
           'Attend.', 'Attend./G']

# Make a dictionary to add "Home" prefix
rename_dict = {col: f'Visitor {col}' for col in columns}
full_data.rename(columns = rename_dict, inplace=True)

#### Feature Engineering

In [268]:
# List of substrings to match for columns to drop
drop_keywords = [
    '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'Rk_x', 'Rk_y',
    'Unnamed', 'Arena', 'Attend.', 'FT/FGA.1', 'FTr', 'FT/FGA', 'eFG%.1',
    'W', 'L', 'PW', 'PL', 'SOS', 'SRS', 'MOV', 'TOV', 'PF', 'TOV%.1',
    'MP', 'G', 'FG', 'FGA', 'ORB', 'DRB', 'TRB']

# Only drop if actual stat (after "Home"/"Visitor") is in drop_keywords
columns_to_keep = []

for col in full_data.columns:
    if col.startswith('Home ') or col.startswith('Visitor '):
        stat_name = col.split(' ', 1)[1]  # split after "Home " or "Visitor "
        if stat_name not in drop_keywords:
            columns_to_keep.append(col)
    else:
        columns_to_keep.append(col)

# Filter the DataFrame
full_data = full_data[columns_to_keep].drop(columns = ['Home Unnamed: 17', 'Home Unnamed: 22', 'Home Unnamed: 27', 'Visitor Unnamed: 17', 'Visitor Unnamed: 22', 'Visitor Unnamed: 27'])



In [269]:
##merge last 10 games record

last10 = pd.read_csv('nba_last10_win.csv')
last10

,Team,L10,L10_W,L10_L,L10_Win_Pct
0,Atlanta Hawks,5-5,5,5,0.5
1,Boston Celtics,8-2,8,2,0.8
2,Brooklyn Nets,3-7,3,7,0.3
3,Charlotte Hornets,1-9,1,9,0.1
4,Chicago Bulls,7-3,7,3,0.7
5,Cleveland Cavaliers,6-4,6,4,0.6
6,Dallas Mavericks,4-6,4,6,0.4
7,Denver Nuggets,5-5,5,5,0.5
8,Detroit Pistons,4-6,4,6,0.4
9,Golden State Warriors,7-3,7,3,0.7


In [270]:
team_abbreviations = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'Charlotte Hornets': 'CHA',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'LA Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}

# Replace for Home and Visitor columns
last10['Team'] = last10['Team'].replace(team_abbreviations)

last10.drop(columns = ['L10', 'L10_W', 'L10_L'], inplace = True)

In [271]:
#merge home last 10 win pct
full_data = pd.merge(full_data, last10, how = 'left', left_on = 'Home', right_on = 'Team').rename(columns = {'L10_Win_Pct' : 'Home Win Pct'})

In [272]:
#merge vis last 10 win pct
full_data = pd.merge(full_data, last10, how = 'left', left_on = 'Visitor', right_on = 'Team').rename(columns = {'L10_Win_Pct' : 'Visitor Win Pct'}).drop(columns = ['Team_x', 'Team_y'])

In [273]:
full_data

,Date,Visitor,VPTS,Home,HPTS,Outcome,Home Team,Home FG%,Home 3P%,Home 2P%,Home FT%,Home AST,Home STL,Home BLK,Home PTS,Home Age,Home ORtg,Home DRtg,Home NRtg,Home Pace,Home 3PAr,Home TS%,Home eFG%,Home TOV%,Home ORB%,Home DRB%,Home Attend./G,Visitor Team,Visitor FG%,Visitor 3P%,Visitor 2P%,Visitor FT%,Visitor AST,Visitor STL,Visitor BLK,Visitor PTS,Visitor Age,Visitor ORtg,Visitor DRtg,Visitor NRtg,Visitor Pace,Visitor 3PAr,Visitor TS%,Visitor eFG%,Visitor TOV%,Visitor ORB%,Visitor DRB%,Visitor Attend./G,Home Win Pct,Visitor Win Pct
0,2024-10-22,NYK,109,BOS,132,W,BOS,0.462,0.368,0.571,0.799,26.1,7.2,5.5,116.3,28.9,120.6,111.1,9.5,95.7,0.536,0.591,0.561,10.8,25.7,76.0,19156,NYK,0.486,0.369,0.558,0.800,27.5,8.2,4.0,115.8,27.5,118.5,114.3,4.2,96.7,0.382,0.589,0.556,11.9,26.0,74.5,19800,0.8,0.6
1,2024-10-22,MIN,103,LAL,110,W,LAL,0.479,0.366,0.562,0.785,26.0,7.7,4.5,113.4,27.9,115.9,114.7,1.2,97.6,0.425,0.593,0.557,12.7,23.3,74.5,18723,MIN,0.468,0.377,0.545,0.789,26.1,8.0,5.0,114.3,27.2,116.6,111.5,5.1,97.3,0.455,0.588,0.554,13.0,25.8,75.1,18835,0.6,0.8
2,2024-10-23,IND,115,DET,109,L,DET,0.476,0.362,0.550,0.774,26.4,8.0,5.3,115.5,25.8,115.0,113.1,1.9,99.8,0.394,0.580,0.547,13.0,26.2,76.0,19071,IND,0.488,0.368,0.568,0.789,29.2,8.5,5.5,117.4,25.8,116.5,114.3,2.2,99.9,0.400,0.594,0.562,11.8,21.3,74.5,16737,0.4,0.8
3,2024-10-23,BKN,116,ATL,120,W,ATL,0.472,0.358,0.552,0.775,29.6,9.7,5.1,118.2,24.9,114.6,115.7,-1.1,102.6,0.410,0.579,0.546,13.2,26.3,76.0,16440,BKN,0.437,0.344,0.515,0.787,25.2,7.8,4.3,105.1,24.7,108.5,115.8,-7.3,96.4,0.458,0.552,0.516,13.7,24.5,75.0,17398,0.5,0.3
4,2024-10-23,ORL,116,MIA,97,L,MIA,0.465,0.367,0.538,0.791,26.4,8.1,3.8,110.6,26.8,113.5,112.9,0.6,96.2,0.429,0.576,0.544,12.4,22.3,75.8,19716,ORL,0.445,0.318,0.533,0.775,23.0,8.9,6.0,105.4,24.9,109.4,109.6,-0.2,96.1,0.411,0.550,0.510,12.9,25.3,77.0,18588,0.6,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,2025-04-15,MEM,116,GSW,121,W,GSW,0.451,0.364,0.528,0.764,29.1,9.4,4.8,113.8,28.6,115.0,111.7,3.3,98.7,0.469,0.568,0.536,12.3,27.3,75.6,18064,MEM,0.479,0.367,0.556,0.786,28.4,8.9,5.6,121.7,24.7,117.7,113.0,4.7,103.3,0.406,0.588,0.554,13.1,28.7,74.9,16660,0.7,0.4
1233,2025-04-16,MIA,109,CHI,90,L,CHI,0.470,0.367,0.556,0.809,29.1,7.6,4.7,117.8,25.2,114.1,115.6,-1.5,102.8,0.457,0.585,0.553,12.7,22.3,76.6,20138,MIA,0.465,0.367,0.538,0.791,26.4,8.1,3.8,110.6,26.8,113.5,112.9,0.6,96.2,0.429,0.576,0.544,12.4,22.3,75.8,19716,0.7,0.6
1234,2025-04-16,DAL,120,SAC,106,L,SAC,0.478,0.357,0.555,0.806,26.5,7.6,4.4,115.7,27.8,116.7,116.2,0.5,98.2,0.391,0.582,0.548,11.8,25.4,76.9,16986,DAL,0.479,0.364,0.552,0.770,25.2,7.8,5.4,114.2,27.5,114.8,116.0,-1.2,99.2,0.389,0.583,0.549,12.5,23.3,73.1,19580,0.5,0.4
1235,2025-04-18,MIA,123,ATL,114,L,ATL,0.472,0.358,0.552,0.775,29.6,9.7,5.1,118.2,24.9,114.6,115.7,-1.1,102.6,0.410,0.579,0.546,13.2,26.3,76.0,16440,MIA,0.465,0.367,0.538,0.791,26.4,8.1,3.8,110.6,26.8,113.5,112.9,0.6,96.2,0.429,0.576,0.544,12.4,22.3,75.8,19716,0.5,0.6


In [284]:
full_data.to_csv('no_diff.csv')

#### Get Feature Differences

In [274]:
def get_differences(df, features, target):
    
    col_list = []
    for feature in features:
        col_name = f'{feature} Difference'
        team1 = f'Home {feature}'
        team2 = f'Visitor {feature}'

        df[col_name] = df[team1] - df[team2]

        col_list.append(col_name)
        
        
    else:
        subset = df.copy()
        
    subset = subset[col_list + [target]]
    #change w a 1 and l a 0
    subset[target] = subset[target].apply(lambda x: 1 if x == 'W' else 0)

    X = subset[col_list]
    y = subset[target]


    return X, y

In [276]:
features = [
       'FG%', '3P%', '2P%', 'FT%', 'AST', 'STL',
       'BLK', 'PTS', 'Age', 'ORtg', 'DRtg',
       'NRtg', 'Pace', '3PAr', 'TS%', 'eFG%',
       'TOV%', 'ORB%', 'DRB%', 'Attend./G',
       'Win Pct']

test = get_differences(full_data, features, 'Outcome')

In [281]:
final = pd.concat([test[0], test[1]], axis=1)

In [283]:
final.to_csv('final_data.csv')